In [1]:
import numpy as np
from pymatgen import Composition, Element
#build NN using tensorflow
#use one hiddlen layer, calculate the error and loss
#split the dataset to about 80% 10% 10%(or less, boundry condition) as training, validation and test set.

In [2]:
#get main group elements list
with open("elements_list_main.txt", "r") as text_file:
    a=text_file.read()
main_group_list=a.split()

In [3]:
#load vectors(from SVD_20)
F=np.loadtxt('SVD_main_group.txt')

In [4]:
#load formation energy as a dictionary
with open("formation_energy_dataset.txt", "r") as text_file:
    diction=text_file.read()
di=diction.split()
formation_dict={}
for i in range(len(di)):
    if i%2==0:
        formation_dict[di[i]]=di[i+1]

In [5]:
#build input vector(20vectors*4elements=80vectors)
#vector_input.shape=[5628,20]
vector_input=np.zeros((len(formation_dict),F.shape[1]*4))

formula_number=0
test_string_list=[]
for formula in formation_dict:
    vector=np.zeros((F.shape[1]*4))
    
    compound=Composition(formula)
    element_list=[]
    for element in compound:
        element_list.append(str(element))
    
    #sort element_list by the number of elements
    element_list_sorted=sorted(element_list,key=lambda s: compound[s])
    #build vector
    #test sort result
    formula_string=""
    element_order=0
    for element in element_list_sorted:
        formula_string+=str(element)+str(int(compound[element]))
        for j in range(len(main_group_list)):
            if str(element)==main_group_list[j]:
                #print(j)
                #print(F[j,:])
                vector[element_order*F.shape[1]:(element_order+1)*F.shape[1]]=F[j,:]
                #print(len(vector))
        element_order+=1
    #print(vector)
    vector_input[formula_number,:]=vector
    test_string_list.append(formula_string)
    formula_number+=1

#build success, input vector=vector_input 5628*80

In [6]:
vector_input.shape

(5628, 80)

In [7]:
#build formula and energy list
formula_list=[]
energy_list=[]
for formula in formation_dict:
    energy_list.append(formation_dict[formula])

for formula in test_string_list:
    formula_list.append(formula)
#build success, energy list=energy_list 5628
#build success, formula list=formula_list 5628
print(len(energy_list))
print(len(formula_list))

5628
5628


In [8]:
##store the dataset in file
#1.store the formula_list and energy_list to formation_energy_formula.txt

with open("FE_TRUE.txt", "w") as text_file:
    for i in range(len(formula_list)):
        text_file.write(formula_list[i]+" "+energy_list[i]+"\n")
        
#2. store the vector_input in SVD_20_INPUT.txt
np.savetxt("SVD_20_INPUT.txt",vector_input)

In [9]:
#build index number list
index_list=list(range(len(formula_list)))
#shuffle index_list
np.random.shuffle(index_list)

split_percent=[0.8,0.1,0.1]
boundry1=int(len(index_list)*split_percent[0])
boundry2=int(len(index_list)*(split_percent[1]+split_percent[0]))

#build split-index matrix
traning_index=index_list[:boundry1]
validation_index=index_list[boundry1:boundry2]
test_index=index_list[boundry2:]

#test split lenth
print(len(traning_index))
print(len(validation_index))
print(len(test_index))

4502
563
563


In [10]:
#vector=vector_input 5628*80
#energy list=energy_list 5628 (string array, to be transformed to numpy array, dtype=np.double)
#formula list=formula_list 5628

In [11]:
training_x=vector_input[traning_index,:]
validation_x=vector_input[validation_index,:]
test_x=vector_input[test_index,:]

#trasform energy_list(string list) to numpy, dtype=np.double
energy_array=np.array(energy_list,dtype=np.double)

training_y=energy_array[traning_index]
validation_y=energy_array[validation_index]
test_y=energy_array[test_index]

#training_x
#training_y
#validation_x
#validation_y
#test_x
#test_y

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
])
model.compile(optimizer='adam',
              loss='mae',
              metrics=['mae', 'acc'])

history=model.fit(training_x, training_y, validation_data=[validation_x,validation_y],epochs=2000)
evaluation=model.evaluate(validation_x, validation_y)

Train on 4502 samples, validate on 563 samples
Epoch 1/2000
4502/4502 [==============================] - 1s 152us/step - loss: 7.8101 - mean_absolute_error: 7.8101 - acc: 0.0204 - val_loss: 7.9949 - val_mean_absolute_error: 7.9949 - val_acc: 0.0337
Epoch 2/2000
4502/4502 [==============================] - 0s 65us/step - loss: 7.5374 - mean_absolute_error: 7.5374 - acc: 0.0522 - val_loss: 7.7717 - val_mean_absolute_error: 7.7717 - val_acc: 0.0870
Epoch 3/2000
4502/4502 [==============================] - 0s 79us/step - loss: 7.3031 - mean_absolute_error: 7.3031 - acc: 0.0802 - val_loss: 7.5900 - val_mean_absolute_error: 7.5900 - val_acc: 0.1101
Epoch 4/2000
4502/4502 [==============================] - 0s 70us/step - loss: 7.1105 - mean_absolute_error: 7.1105 - acc: 0.0951 - val_loss: 7.4304 - val_mean_absolute_error: 7.4304 - val_acc: 0.1226
Epoch 5/2000
4502/4502 [==============================] - 0s 70us/step - loss: 6.9486 - mean_absolute_error: 6.9486 - acc: 0.1037 - val_loss: 7.2940

4502/4502 [==============================] - 0s 44us/step - loss: 5.1588 - mean_absolute_error: 5.1588 - acc: 0.1544 - val_loss: 5.6503 - val_mean_absolute_error: 5.6503 - val_acc: 0.1456
Epoch 42/2000
4502/4502 [==============================] - 0s 40us/step - loss: 5.1375 - mean_absolute_error: 5.1375 - acc: 0.1502 - val_loss: 5.6307 - val_mean_absolute_error: 5.6307 - val_acc: 0.1403
Epoch 43/2000
4502/4502 [==============================] - 0s 44us/step - loss: 5.1168 - mean_absolute_error: 5.1168 - acc: 0.1493 - val_loss: 5.6125 - val_mean_absolute_error: 5.6125 - val_acc: 0.1439
Epoch 44/2000
4502/4502 [==============================] - 0s 64us/step - loss: 5.0969 - mean_absolute_error: 5.0969 - acc: 0.1502 - val_loss: 5.5945 - val_mean_absolute_error: 5.5945 - val_acc: 0.1492
Epoch 45/2000
4502/4502 [==============================] - 0s 66us/step - loss: 5.0769 - mean_absolute_error: 5.0769 - acc: 0.1635 - val_loss: 5.5771 - val_mean_absolute_error: 5.5771 - val_acc: 0.1563
Epoc

4502/4502 [==============================] - 0s 92us/step - loss: 4.5641 - mean_absolute_error: 4.5641 - acc: 0.2052 - val_loss: 5.0907 - val_mean_absolute_error: 5.0907 - val_acc: 0.2060
Epoch 82/2000
4502/4502 [==============================] - 0s 83us/step - loss: 4.5535 - mean_absolute_error: 4.5535 - acc: 0.2059 - val_loss: 5.0811 - val_mean_absolute_error: 5.0811 - val_acc: 0.2131
Epoch 83/2000
4502/4502 [==============================] - 0s 107us/step - loss: 4.5433 - mean_absolute_error: 4.5433 - acc: 0.2150 - val_loss: 5.0710 - val_mean_absolute_error: 5.0710 - val_acc: 0.2220
Epoch 84/2000
4502/4502 [==============================] - 0s 97us/step - loss: 4.5330 - mean_absolute_error: 4.5330 - acc: 0.2252 - val_loss: 5.0613 - val_mean_absolute_error: 5.0613 - val_acc: 0.2256
Epoch 85/2000
4502/4502 [==============================] - 0s 99us/step - loss: 4.5231 - mean_absolute_error: 4.5231 - acc: 0.2283 - val_loss: 5.0522 - val_mean_absolute_error: 5.0522 - val_acc: 0.2202
Epo

4502/4502 [==============================] - 0s 100us/step - loss: 4.2601 - mean_absolute_error: 4.2601 - acc: 0.2341 - val_loss: 4.7686 - val_mean_absolute_error: 4.7686 - val_acc: 0.2327
Epoch 122/2000
4502/4502 [==============================] - 1s 123us/step - loss: 4.2545 - mean_absolute_error: 4.2545 - acc: 0.2401 - val_loss: 4.7620 - val_mean_absolute_error: 4.7620 - val_acc: 0.2433
Epoch 123/2000
4502/4502 [==============================] - 1s 122us/step - loss: 4.2494 - mean_absolute_error: 4.2494 - acc: 0.2521 - val_loss: 4.7558 - val_mean_absolute_error: 4.7558 - val_acc: 0.2558
Epoch 124/2000
4502/4502 [==============================] - 0s 104us/step - loss: 4.2441 - mean_absolute_error: 4.2441 - acc: 0.2481 - val_loss: 4.7496 - val_mean_absolute_error: 4.7496 - val_acc: 0.2504
Epoch 125/2000
4502/4502 [==============================] - 0s 111us/step - loss: 4.2388 - mean_absolute_error: 4.2388 - acc: 0.2463 - val_loss: 4.7435 - val_mean_absolute_error: 4.7435 - val_acc: 0.

4502/4502 [==============================] - 0s 67us/step - loss: 4.0948 - mean_absolute_error: 4.0948 - acc: 0.2723 - val_loss: 4.5667 - val_mean_absolute_error: 4.5667 - val_acc: 0.2735
Epoch 162/2000
4502/4502 [==============================] - 0s 69us/step - loss: 4.0916 - mean_absolute_error: 4.0916 - acc: 0.2763 - val_loss: 4.5633 - val_mean_absolute_error: 4.5633 - val_acc: 0.2735
Epoch 163/2000
4502/4502 [==============================] - 0s 74us/step - loss: 4.0884 - mean_absolute_error: 4.0884 - acc: 0.2837 - val_loss: 4.5600 - val_mean_absolute_error: 4.5600 - val_acc: 0.2824
Epoch 164/2000
4502/4502 [==============================] - 0s 68us/step - loss: 4.0854 - mean_absolute_error: 4.0854 - acc: 0.2863 - val_loss: 4.5569 - val_mean_absolute_error: 4.5569 - val_acc: 0.2931
Epoch 165/2000
4502/4502 [==============================] - 0s 69us/step - loss: 4.0825 - mean_absolute_error: 4.0825 - acc: 0.2932 - val_loss: 4.5532 - val_mean_absolute_error: 4.5532 - val_acc: 0.3020


...

4502/4502 [==============================] - 0s 67us/step - loss: 3.7222 - mean_absolute_error: 3.7222 - acc: 0.3556 - val_loss: 4.1874 - val_mean_absolute_error: 4.1874 - val_acc: 0.3925
Epoch 1242/2000
4502/4502 [==============================] - 0s 62us/step - loss: 3.7220 - mean_absolute_error: 3.7220 - acc: 0.3590 - val_loss: 4.1876 - val_mean_absolute_error: 4.1876 - val_acc: 0.4103
Epoch 1243/2000
4502/4502 [==============================] - 0s 63us/step - loss: 3.7219 - mean_absolute_error: 3.7219 - acc: 0.3629 - val_loss: 4.1871 - val_mean_absolute_error: 4.1871 - val_acc: 0.3925
Epoch 1244/2000
4502/4502 [==============================] - 0s 65us/step - loss: 3.7218 - mean_absolute_error: 3.7218 - acc: 0.3587 - val_loss: 4.1869 - val_mean_absolute_error: 4.1869 - val_acc: 0.3872
Epoch 1245/2000
4502/4502 [==============================] - 0s 68us/step - loss: 3.7216 - mean_absolute_error: 3.7216 - acc: 0.3470 - val_loss: 4.1867 - val_mean_absolute_error: 4.1867 - val_acc: 0.3

4502/4502 [==============================] - 0s 66us/step - loss: 3.7175 - mean_absolute_error: 3.7175 - acc: 0.3538 - val_loss: 4.1826 - val_mean_absolute_error: 4.1826 - val_acc: 0.3961
Epoch 1282/2000
4502/4502 [==============================] - 0s 66us/step - loss: 3.7176 - mean_absolute_error: 3.7176 - acc: 0.3478 - val_loss: 4.1831 - val_mean_absolute_error: 4.1831 - val_acc: 0.3890
Epoch 1283/2000
4502/4502 [==============================] - 0s 65us/step - loss: 3.7173 - mean_absolute_error: 3.7173 - acc: 0.3476 - val_loss: 4.1824 - val_mean_absolute_error: 4.1824 - val_acc: 0.3908
Epoch 1284/2000
4502/4502 [==============================] - 0s 70us/step - loss: 3.7172 - mean_absolute_error: 3.7172 - acc: 0.3456 - val_loss: 4.1826 - val_mean_absolute_error: 4.1826 - val_acc: 0.3925
Epoch 1285/2000
4502/4502 [==============================] - 0s 65us/step - loss: 3.7171 - mean_absolute_error: 3.7171 - acc: 0.3598 - val_loss: 4.1827 - val_mean_absolute_error: 4.1827 - val_acc: 0.4

4502/4502 [==============================] - 0s 63us/step - loss: 3.7131 - mean_absolute_error: 3.7131 - acc: 0.3550 - val_loss: 4.1791 - val_mean_absolute_error: 4.1791 - val_acc: 0.4032
Epoch 1322/2000
4502/4502 [==============================] - 0s 66us/step - loss: 3.7130 - mean_absolute_error: 3.7130 - acc: 0.3592 - val_loss: 4.1788 - val_mean_absolute_error: 4.1788 - val_acc: 0.4085
Epoch 1323/2000
4502/4502 [==============================] - 0s 63us/step - loss: 3.7129 - mean_absolute_error: 3.7129 - acc: 0.3652 - val_loss: 4.1792 - val_mean_absolute_error: 4.1792 - val_acc: 0.4085
Epoch 1324/2000
4502/4502 [==============================] - 0s 69us/step - loss: 3.7128 - mean_absolute_error: 3.7128 - acc: 0.3598 - val_loss: 4.1789 - val_mean_absolute_error: 4.1789 - val_acc: 0.4085
Epoch 1325/2000
4502/4502 [==============================] - 0s 64us/step - loss: 3.7127 - mean_absolute_error: 3.7127 - acc: 0.3627 - val_loss: 4.1783 - val_mean_absolute_error: 4.1783 - val_acc: 0.4

4502/4502 [==============================] - 0s 63us/step - loss: 3.7087 - mean_absolute_error: 3.7087 - acc: 0.3647 - val_loss: 4.1751 - val_mean_absolute_error: 4.1751 - val_acc: 0.3996
Epoch 1362/2000
4502/4502 [==============================] - 0s 67us/step - loss: 3.7085 - mean_absolute_error: 3.7085 - acc: 0.3514 - val_loss: 4.1744 - val_mean_absolute_error: 4.1744 - val_acc: 0.3961
Epoch 1363/2000
4502/4502 [==============================] - 0s 64us/step - loss: 3.7083 - mean_absolute_error: 3.7083 - acc: 0.3496 - val_loss: 4.1745 - val_mean_absolute_error: 4.1745 - val_acc: 0.3979
Epoch 1364/2000
4502/4502 [==============================] - 0s 64us/step - loss: 3.7083 - mean_absolute_error: 3.7083 - acc: 0.3598 - val_loss: 4.1743 - val_mean_absolute_error: 4.1743 - val_acc: 0.4014
Epoch 1365/2000
4502/4502 [==============================] - 0s 71us/step - loss: 3.7081 - mean_absolute_error: 3.7081 - acc: 0.3554 - val_loss: 4.1744 - val_mean_absolute_error: 4.1744 - val_acc: 0.3

4502/4502 [==============================] - 0s 65us/step - loss: 3.7035 - mean_absolute_error: 3.7035 - acc: 0.3809 - val_loss: 4.1687 - val_mean_absolute_error: 4.1687 - val_acc: 0.4210
Epoch 1402/2000
  32/4502 [..............................] - ETA: 0s - loss: 4.2217 - mean_absolute_error: 4.2217 - acc: 0.4062

In [14]:
model.evaluate(validation_x, validation_y)

563/563 [==============================] - 0s 210us/step


[4.280741606046632, 4.280741606046632, 0.3836589704398364]

In [15]:
model.evaluate(test_x, test_y)

563/563 [==============================] - 0s 35us/step


[3.7028818342232577, 3.7028818342232577, 0.35879218430121035]

In [ ]:
'''
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
# Fit the model
model.fit(X, Y, validation_split=0.33, epochs=150, batch_size=10, callbacks=callbacks_list, verbose=0)
'''